<b> 
Parses Schema Compare scripts output from both:

    AWS SCT (schema conversion tool - can work on multi db-engines)
    AzDS SC (sql db comparison only)
</b>
The ideal is use AWS SCT  [Source System] vs [empty AWS Target] returning:

    -tables 
    -unique business keys (not surrogate keys) which can be used for MERGE statements
<b>
Having a central DataOps DB Instance offers:
</b>

    -potential for cross-cloud mgt via setupSyn method
    -BDD ownership

In [1]:
import sys
import os
import json

if 'DataOps/py/dep' not in os.getcwd():
    os.chdir('dep')

from shutil import copyfile
from FileOps import fs

if 'ipy' in sys.argv[0]:
    with open('../.vscode/launch.json', 'r') as config_file:
        config_dict = json.load(config_file)
    _arg = config_dict["configurations"][0]["linux"]["args"]
else:
    _arg = sys.argv[1:] #need to strip [0] so matches launch.json

In [2]:
# Constants
pthFW = _arg[3]

In [3]:
# Vars
srcType = 'aws'

db = 'realestate'
fwStage = 'realestate_init/fwStage'

lObjVar = [4, 'c', 'ALTER']            # Only Run through 1 file at a time
lObjIndex =     lObjVar[0]         
objType =       lObjVar[1]  
opAndsearchType=lObjVar[2]  

createTable = False
createStage = False

In [21]:
if srcType == 'aws':
    lMin = 1    #Fortunately the lMin and lMax values here are same for all AWS files
    lMax = -1
    pthSCT = '/media/paul/code/db/awsSCT/realestate/Microsoft SQL Server-20190814-20-40-11'
    lObj = ['0. drop-table.sql', '1. create-schema.sql', '2. create-table.sql', '3. create-index.sql', '4. create-constraint.sql']
    fle = lObj[lObjIndex]
    l1End = ']('        #line 1 end  is used to parse obj name in first line - needed for create-table only
elif srcType == 'az':
    lMin = 8
    lMax = -2
    pthSCT = '/media/paul/code/db/azSC/realestate'
    fle = 'azTextSchemas.sql'
    l1End = '] ('

In [22]:
def tableStgVer(s, iSkip):
    lList = s.splitlines()
    sql = lList[0] + '\n'
    for i, l in enumerate(lList[1:]): 
        if i + iSkip < len(lList):
            sql = sql + l[l.index('['):l.index(']')+1] + ' nvarchar(255) NULL,\n'
        elif i + iSkip == len(lList):
            sql = sql[:-2] + '\n' + l
    return sql

In [23]:
fwFS = fs()

# First deal with splitting Tables into Individual files
srcPath = os.path.join(pthSCT, fle)    
fileList = open(srcPath,'r').read().split('GO')

In [25]:

for num, file in enumerate(fileList[lMin:lMax]):     #other statements not in this range are not Table Creates

    if createTable:
        if file[1:13] == 'CREATE TABLE' or srcType == 'aws':

            fleName = file[file.index('['):file.index(l1End)].replace('[', '').replace(']', '')   

            if srcType == 'aws': 
                sql = file[file.index('CREATE TABLE'):file.index(')\n') + 1] + ';'
            elif srcType == 'az':
                sql = file[file.index('CREATE TABLE'):file.index(');') + 2]
    else:
        #fleName = file[file.index('['):file.index('\n') ].replace('[', '').replace(']', '')
        sql = file[file.index(opAndsearchType):]
        fleName = sql[sql.index('['):sql.index('\n') ].replace('[', '').replace(']', '')

    
    fwPath = fwFS.genFlywayFullPath(pthRoot = pthFW, dbs = fwStage, objName = fleName, fleType = objType, opType = opAndsearchType)
    open(fwPath, 'w').write(sql)  
    
    if createStage:
        sql = tableStgVer(sql, 2)
        fwPath = fwPath.replace('dbo','stg')
        open(fwPath, 'w').write(sql.replace('dbo','stg'))
        fwRevMax = fwRevMax + 1

In [8]:
#We are now splitting all object types and doing so 1 file at a time so this cell not needed
#if srcType == 'aws':            
    # Now just name, version and copy remainder as is
    #for i in lObj:
       #a = i.split('.')
        #srcPath = os.path.join(pthSCT, i)
        #fwPath = fwFS.genFlywayFullPath(pthRoot = pthFW, dbs = fwStage, objName = a[1].strip(), fleType = 's', opType = 'Script') #, revision = a[0])
        #copyfile(srcPath, fwPath)

In [9]:
file.split('\n')[3]

'CREATE NONCLUSTERED INDEX [flyway_schema_history_s_idx]'

In [10]:
sql[sql.index('['):sql.index('\n') ].replace('[', '').replace(']', '')

'flyway_schema_history_s_idx'

In [5]:
pwd

'/media/paul/code/Python/DataOps/py/dep'